# THai AI - Cause of Dead(THAI-COD)
##### For THAIBOD
##### By Dr. Thanawat Wongphan
##### V. 1.0 Indy Code name
##### V. 1.1 Trat version, Real data with cleaning
##### Contact: thwong@iu.edu

### <font color=orange>Section I: Introduction</font>
This program is an innovation which has been well designed for predict best suitable cause of dead(COD) by using multiple reasons from hospital summary. Using the training set from 2020 research from Thai BOD. This program has been designed by python infrastructure with AI abilities from tensorflow opensource. This program will compare multiple AI-models and return which types of model should be selected in Thailand setting.

#### This programs consisting of six sections:
* Section I:    <tab>Introduction</tab>
* Section II:   <tab>library and functions load</tab>
* Section III:   <tab>Data manipulation</tab>
* Section IV:  AI process
* Section V:   Analysis
* Section VI:    Conclusion and discussion

<b>This jupyter notebook can be used and tested by open communities under the GPL-licensed by Dr.Thanawat Wongphan and THAI-BOD working group.</b>



### <font color=orange>Section II: Library and functions load</font>

In [302]:
## Section II:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Dense, Input, Dropout, LSTM, Activation, Conv1D,MaxPooling1D,Flatten,GlobalAveragePooling1D,GlobalMaxPooling1D
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.initializers import glorot_uniform
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras import layers
from keras.layers import Flatten
from keras import backend as K

from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import MinMaxScaler,StandardScaler,LabelEncoder
from sklearn.metrics import mean_squared_error,accuracy_score, confusion_matrix, precision_score, recall_score, ConfusionMatrixDisplay
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from scipy.stats import randint

# Tree Visualisation
from sklearn.tree import export_graphviz
from IPython.display import Image
import graphviz

### <font color=orange>Section III: Data manipulation</font>

3.1 Data loading:

In [303]:
flname="Data/SampleData.xlsx"
OriginalCSV = pd.read_excel(flname)
print("File: {} has {} columns and {} rows. The sample of first three rows are:".format(flname,OriginalCSV.shape[1],OriginalCSV.shape[0]))
OriginalCSV.head(3)

File: Data/SampleData.xlsx has 40 columns and 6162 rows. The sample of first three rows are:


,qcode,Province,Province_name,sex,age,lccaattmm,dyear,dmon,ddate,ghos_name,...,P_a_Final_New,P_a_ICD10_Final_New,P_b_Final_New,P_b_ICD10_Final_New,P_c_Final_New,P_c_ICD10_Final_new,P_d_Final_New,P_d_ICD10_Final_New,P_e_Final_New,P_e_ICD10_Final_New
0,1102010011,10,BKK,1,0,10200400,2561,12,24,IN,...,HYDRANENCEPHALY WITH SEVERE HYDROCEL WITH RDS,"Q043,P835",PRETERM,-,TEENAGE PREGNANCY,NaN,,NaN,,NaN
1,1102010021,10,BKK,1,0,10200400,2561,12,9,IN,...,CONGENITAL HEART DIS. (PULMONARY VALVE ATRESIA...,"Q220,Q250,P008,P832","RDS WITH VVLBE, PRETERM",P220,MATERNAL GDMA,O244,,NaN,,NaN
2,1102010031,10,BKK,2,0,10200400,2561,12,3,IN,...,CONGENITAL HYPOPLASIA LEFT HEART SYNDROME(ASD ...,"Q234,Q211",LBW,P070,,NaN,,NaN,,NaN


3.2 Data cleaning and text labels encoding

In [304]:
CleanData=OriginalCSV
labels_ICDs = LabelEncoder()
Corpus =    list(set(list(CleanData['Final_dx_New_ICD10']))) + list(set(list(CleanData['a_ICD10_Final_New']))) + list(set(list(CleanData['b_ICD10_Final_New']))) + list(set(list(CleanData['c_ICD10_Final_New']))) + list(set(list(CleanData['d_ICD10_Final_New']))) 
labels_ICDs.fit(Corpus)
CleanData['a_ICD10_Enc']=labels_ICDs.transform(CleanData['a_ICD10_Final_New'].values.ravel())
CleanData['b_ICD10_Enc']=labels_ICDs.transform(CleanData['b_ICD10_Final_New'].values.ravel())
CleanData['c_ICD10_Enc']=labels_ICDs.transform(CleanData['c_ICD10_Final_New'].values.ravel())
CleanData['d_ICD10_Enc']=labels_ICDs.transform(CleanData['d_ICD10_Final_New'].values.ravel())
CleanData['Final_dx_Enc']=labels_ICDs.transform(CleanData['Final_dx_New_ICD10'].values.ravel())
ConversionTable=pd.read_excel("Data/All_Units_Conversion.xlsx")
def TimeStandardize(tmp):
    if (isinstance(tmp, str)):
        if((tmp.find(" ")>-1) & (tmp.find(",")==-1) & (tmp.find("-")==-1) & (tmp.find("last")==-1) ):
            t=tmp.split(" ")
            t[0]=t[0].replace("+",'')
            t[0]=t[0].replace("ๅ",'')
            t[0]=t[0].replace("<",'')
            t[0]=t[0].replace("At",'')
            t[0]=t[0].replace("?",'')
            if((t[0]=='') | ((t[0]=='.'))):
                return np.nan
            else:
                try:
                    V1=ConversionTable[ConversionTable['Unit']==t[1]]['Value'].iloc[0]
                    V0=float(t[0])
                    MultiplyResult = V0*V1
                    return MultiplyResult
                except:
                    return np.nan
        else:
            return np.nan
    else:
        return np.nan
CleanData['a_time_standard']=CleanData['a_time_Final_New'].apply(TimeStandardize)
CleanData['b_time_standard']=CleanData['b_time_Final_New'].apply(TimeStandardize)
CleanData['c_time_standard']=CleanData['c_time_Final_New'].apply(TimeStandardize)
CleanData['d_time_standard']=CleanData['d_time_Final_New'].apply(TimeStandardize)
CleanData.dropna(subset=['a_time_standard', 'b_time_standard','c_time_standard','d_time_standard'])
CleanData['place']=CleanData['ghos_name'].apply(lambda x: 1 if x=='IN' else 0)

3.3 Data shuffle and partitioning

3.3.1 Sizing partition

In [305]:
#Declare ratio of Train/Dev and the rest is test set
TrainRatio=0.8
DevRatio=0.1
TrainSize=int(np.round(TrainRatio*CleanData.shape[0]))
DevSize=int(np.round(DevRatio*CleanData.shape[0]))
TestSize=CleanData.shape[0]-TrainSize-DevSize
print("From total {:,} rows, the samples have been partitioned into 3 sets: Train/Dev/Test size: {}/{}/{}".format(
    CleanData.shape[0],TrainSize,DevSize,TestSize
))

From total 6,162 rows, the samples have been partitioned into 3 sets: Train/Dev/Test size: 4930/616/616


3.3.2 Shuffle index

In [306]:
ind=CleanData.index
ind_list=ind.to_list()
np.random.shuffle(ind_list)
ind_shuffle_list=ind_list
TrainIndex=ind_shuffle_list[0:TrainSize]
DevIndex=ind_shuffle_list[TrainSize:(TrainSize+DevSize)]
TestIndex=ind_shuffle_list[(TrainSize+DevSize):]

3.3.3 partition data by index


In [307]:
TrainSet=CleanData.iloc[TrainIndex]
DevSet=CleanData.iloc[DevIndex]
TestSet=CleanData.iloc[TestIndex]

### <font color=orange>Section IV: AI</font>

##### 4.0 Random Forest Explore

In [308]:
TrainSet.columns

Index(['qcode', 'Province', 'Province_name', 'sex', 'age', 'lccaattmm',
       'dyear', 'dmon', 'ddate', 'ghos_name', 'dcause_1', 'ncause',
       'agreement_New', 'Final_dx_New', 'Final_dx_New_ICD10', 'a_Final_new',
       'a_ICD10_Final_New', 'a_time_Final_New', 'b_Final_New',
       'b_ICD10_Final_New', 'b_time_Final_New', 'c_Final_New',
       'c_ICD10_Final_New', 'c_time_Final_New', 'd_Final_New',
       'd_ICD10_Final_New', 'd_time_Final_New', 'II_Final_New',
       'II_time_Final_New', 'Rules apply_Final_New', 'P_a_Final_New',
       'P_a_ICD10_Final_New', 'P_b_Final_New', 'P_b_ICD10_Final_New',
       'P_c_Final_New', 'P_c_ICD10_Final_new', 'P_d_Final_New',
       'P_d_ICD10_Final_New', 'P_e_Final_New', 'P_e_ICD10_Final_New',
       'a_ICD10_Enc', 'b_ICD10_Enc', 'c_ICD10_Enc', 'd_ICD10_Enc',
       'Final_dx_Enc', 'a_time_standard', 'b_time_standard', 'c_time_standard',
       'd_time_standard', 'place'],
      dtype='object')

In [309]:
AllColumns = ['qcode', 'Province', 'Province_name', 'sex', 'age', 'lccaattmm',
       'dyear', 'dmon', 'ddate', 'ghos_name', 'dcause_1', 'ncause',
       'agreement_New', 'Final_dx_New', 'Final_dx_New_ICD10', 'a_Final_new',
       'a_ICD10_Final_New', 'a_time_Final_New', 'b_Final_New',
       'b_ICD10_Final_New', 'b_time_Final_New', 'c_Final_New',
       'c_ICD10_Final_New', 'c_time_Final_New', 'd_Final_New',
       'd_ICD10_Final_New', 'd_time_Final_New', 'II_Final_New',
       'II_time_Final_New', 'Rules apply_Final_New', 'P_a_Final_New',
       'P_a_ICD10_Final_New', 'P_b_Final_New', 'P_b_ICD10_Final_New',
       'P_c_Final_New', 'P_c_ICD10_Final_new', 'P_d_Final_New',
       'P_d_ICD10_Final_New', 'P_e_Final_New', 'P_e_ICD10_Final_New',
       'a_ICD10_Enc', 'b_ICD10_Enc', 'd_ICD10_Enc', 'Final_dx_Enc',
       'c_ICD10_Enc', 'NEwCol', 'a_time_standard', 'b_time_standard',
       'c_time_standard', 'd_time_standard', 'place']
ySelectedFeatures = ['Final_dx_Enc']
xSelectedFeatures = ['Province',  'sex', 'age','place',
       'a_ICD10_Enc', 'a_time_standard', 
       'b_ICD10_Enc', 'b_time_standard', 
       'c_ICD10_Enc', 'c_time_standard',
       'd_ICD10_Enc', 'd_time_standard']
"""xSelectedFeatures = ['Province',  'sex', 'age',
       'a_ICD10_Enc',  
       'b_ICD10_Enc', 
       'c_ICD10_Enc', 
       'd_ICD10_Enc']"""

"xSelectedFeatures = ['Province',  'sex', 'age',\n       'a_ICD10_Enc',  \n       'b_ICD10_Enc', \n       'c_ICD10_Enc', \n       'd_ICD10_Enc']"

In [310]:

TrainSet = TrainSet.fillna(0)
DevSet = DevSet.fillna(0)
x_Train=TrainSet[xSelectedFeatures]
y_Train=TrainSet[ySelectedFeatures]
x_Dev=DevSet[xSelectedFeatures]
y_Dev=DevSet[ySelectedFeatures]
clf = RandomForestClassifier(n_estimators = 100)
clf.fit(x_Train, y_Train)
# performing predictions on the test dataset
y_pred = clf.predict(x_Dev)
AccModel=metrics.accuracy_score(y_pred, y_Dev)
print("ACCURACY OF THE MODEL:", AccModel)
BestAccuracy={'Model':['Randomforest'],'Accuracy':AccModel}

/Users/thanawat/anaconda3/envs/spyder/lib/python3.11/site-packages/sklearn/base.py:1152: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


ACCURACY OF THE MODEL: 0.43993506493506496


4.1 CNN Deep learning:

In [312]:
#Use Class at 5 min: 0 = neutral, 1 = sell, 2= buy ,3 = both sell and buy correct

x_Train=TrainSet[xSelectedFeatures]
y_Train_pre_HOT = TrainSet[ySelectedFeatures]
y_Train= tf.one_hot(np.reshape(y_Train_pre_HOT,x_Train.shape[0]),y_Train_pre_HOT[ySelectedFeatures[0]].nunique())
y_Dev = DevSet[ySelectedFeatures]
y_Test = TestSet[ySelectedFeatures][:TestSize]
y_Train_pre_HOT.value_counts()
filtered_train = np.reshape(x_Train,(x_Train.shape[0],x_Train.shape[1],1))
EpochNum=10
filtersize=10
model_cnn_wide_10 = Sequential()
model_cnn_wide_10.add(Conv1D(filters=filtersize, kernel_size=filtered_train.shape[1]-1, input_shape=(filtered_train.shape[1], filtered_train.shape[2])))
model_cnn_wide_10.add(MaxPooling1D(pool_size=1 ))
model_cnn_wide_10.add(Flatten())
model_cnn_wide_10.add(Dense(500, activation='relu'))
model_cnn_wide_10.add(Dense(y_Train_pre_HOT[ySelectedFeatures[0]].nunique(), activation='softmax'))
model_cnn_wide_10.compile(loss='CategoricalCrossentropy', optimizer='adam', metrics=['accuracy'])
model_cnn_wide_10.summary()
history_cnn_wide_10=model_cnn_wide_10.fit(x=filtered_train, y=y_Train, epochs=EpochNum)
yhat_cnn_wide_10 = model_cnn_wide_10.predict(filtered_train)


Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_3 (Conv1D)           (None, 2, 10)             120       
                                                                 
 max_pooling1d_3 (MaxPooling  (None, 2, 10)            0         
 1D)                                                             
                                                                 
 flatten_3 (Flatten)         (None, 20)                0         
                                                                 
 dense_6 (Dense)             (None, 500)               10500     
                                                                 
 dense_7 (Dense)             (None, 714)               357714    
                                                                 
Total params: 368,334
Trainable params: 368,334
Non-trainable params: 0
________________________________________________

In [313]:
#Based on train data:
print("The best accuracy of CNN is {:.3%}".format(history_cnn_wide_10.history['accuracy'][-1]))
BestAccuracy={'Model':['RandomForest','CNN'],'Accuracy':[AccModel,history_cnn_wide_10.history['accuracy'][-1]]}

The best accuracy of CNN is 0.690%


### <font color=orange>Section V: Analysis</font>

5.1 Analysis of accuracy

In [314]:
AccuracyTable = pd.DataFrame({'Model Number':[1,2],
                              'Model Name':[BestAccuracy['Model'][0],BestAccuracy['Model'][1]],
                              'Accuracy':[BestAccuracy['Accuracy'][0],BestAccuracy['Accuracy'][1]]})
display(AccuracyTable )

,Model Number,Model Name,Accuracy
0,1,RandomForest,0.439935
1,2,CNN,0.006897


### <font color=orange>Section VI: Conclusion and discussion</font>

In [315]:
print("""
      As you see, the predictive accuracy from model Random Forest is much higher than CNN but too far from truth.
      Therefore we have to improve model by alpha engineering and model developing.""")


      As you see, the predictive accuracy from model Random Forest is much higher than CNN but too far from truth.
      Therefore we have to improve model by alpha engineering and model developing.
